## <b><font color='darkblue'>Preface</font></b>
([source](https://towardsdatascience.com/clear-intro-to-mcp/)) <font size='3ptx'><b>MCP (Model Context Protocol) is a way to democratize access to tools for AI Agents. In this article we cover the fundamental components of MCP, how they work together, and a code example of how MCP works in practice.</b> As the race to move AI agents from prototype to production heats up, the need for a standardized way for agents to call tools across different providers is pressing. </font>

This transition to a standardized approach to agent tool calling is similar to what we saw with REST APIs. <b>Before they existed, developers had to deal with a mess of proprietary protocols just to pull data from different services. REST brought order to chaos, enabling systems to talk to each other in a consistent way</b>.

<b><font size='3ptx'>[MCP](https://modelcontextprotocol.io/introduction) (Model Context Protocol)</font> is aiming to, as it sounds, provide context for AI models in a standard way</b>. Without it, we’re headed towards tool-calling mayhem where multiple incompatible versions of “standardized” tool calls crop up simply because there’s no shared way for agents to organize, share, and invoke tools. MCP gives us a shared language and the democratization of tool calling.

One thing I’m personally excited about is <b>how tool-calling standards like MCP can actually make [Ai Systems](https://towardsdatascience.com/tag/ai-systems/) safer. With easier access to well-tested tools more companies can avoid reinventing the wheel</b>, which reduces security risks and minimizes the chance of malicious code. As Ai systems start scaling in 2025, these are valid concerns.

As I dove into MCP, I <b>realized a huge gap in documentation. There’s plenty of high-level “what does it do” content, but when you actually want to understand how it works</b>, the resources start to fall short—especially for those who aren’t native developers. It’s either high level explainers or deep in the source code.

<b>In this piece, I’m going to break MCP down for a broader audience — making the concepts and functionality clear and digestible</b>. If you’re able, follow along in the coding section, if not it will be well explained in natural language above the code snippets.

### <b><font color='darkgreen'>An Analogy to Understand MCP: The Restaurant</font></b>
Let’s imagine the concept of MCP as a restaurant where we have:
* **The Host** = The restaurant building (the environment where the agent runs)
* **The Server** = The kitchen (where tools live)
* **The Client** = The waiter (who sends tool requests)
* **The Agent** = The customer (who decides what tool to use)
* **The Tools** = The recipes (the code that gets executed)

### <b><font color='darkgreen'>The Components of MCP</font></b>


#### <b><font size='3ptx'>Host</font></b>
This is where the agent operates. In our analogy, it’s the restaurant building; in MCP, it’s wherever your agents or LLMs actually run. If you’re using Ollama locally, you’re the host. If you’re using Claude or GPT, then Anthropic or OpenAI are the hosts.

#### <b><font size='3ptx'>Client</font></b>
This is the environment that sends tool call requests from the agent. Think of it as the waiter who takes your order and delivers it to the kitchen. In practical terms, it’s the application or interface where your agent runs. The client passes tool call requests to the Server using MCP.

#### <b><font size='3ptx'>Server</font></b>
This is the kitchen where recipes, or tools, are housed. It centralizes tools so agents can access them easily. Servers can be local (spun up by users) or remote (hosted by companies offering tools). Tools on a server are typically either grouped by function or integration. For instance, all Slack-related tools can be on a “Slack server,” or all messaging tools can be grouped together on a “messaging server”. That decision is based on architectural and developer preferences.

#### <b><font size='3ptx'>Agent</font></b>
<b>The “brains” of the operation. Powered by an LLM, it decides which tools to call to complete a task</b>. When it determines a tool is needed, it initiates a request to the server. The agent doesn’t need to natively understand MCP because it learns how to use it through the metadata associated with each of the tools. This metadata associated with each tool tells the agent the protocol for calling the tool and the execution method. But it is important to note that the platform or agent needs to support MCP so that it handles tool calls automatically. Otherwise it is up to the developer to write the complex translation logic of how to parse the metadata from the schema, form tool call requests in MCP format, map the requests to the correct function, execute the code, and return the result in MCP complaint format back to the agent.

#### <b><font size='3ptx'>Tools</font></b>
These are the functions, such as calling APIs or custom code, that “does the work”. Tools live on servers and can be:>
* Custom tools you create and host on a local server.
* Premade tools hosted by others on a remote server.
* Premade code created by others but hosted by you on a local server.

## <b><font color='darkblue'>How the components fit together</font></b>

1. <font size='3ptx'><b>Server Registers Tools</b></font>
> Each tool is defined with a name, description, input/output schemas, a function handler (the code that runs) and registered to the server. This usually involves calling a method or API to tell the server “hey, here’s a new tool and this is how you use it”.

2. <font size='3ptx'><b>Server Exposes Metadata</b></font>
> When the server starts or an agent connects, it exposes the tool metadata (schemas, descriptions) via MCP.

3. <font size='3ptx'><b>Agent Discovers Tools</b></font>
> The agent queries the server (using MCP) to see what tools are available. It understands how to use each tool from the tool metadata. This typically happens on startup or when tools are added.

4. <font size='3ptx'><b>Agent Plans Tool Use</b></font>
> When the agent determines a tool is needed (based on user input or task context), it forms a tool call request in a standardized MCP JSON format which includes tool name, input parameters that match the tool’s input schema, and any other metadata. The client acts as the transport layer and sends the MCP formatted request to the server over HTTP.

5. <font size='3ptx'><b>Translation Layer Executes</b></font>
> The translation layer takes the agent’s standardized tool call (via MCP), maps the request to the corresponding function on the server, executes the function, formats the result back to MCP, and sends it back to the agent. A framework that abstracts MCP for you deos all of this without the developer needing to write the translation layer logic (which sounds like a headache).

![procedure call](https://contributor.insightmediagroup.io/wp-content/uploads/2025/03/AD_4nXeeW11YxoD4PFmiDyq3U05WG_plNlrzy7IZUa1bcWQfG2_ECnwJ3xlGSFGMX94R_f0ZwmrZyqUBu4u-uIfvwiJD74XKtCu94FTE0vkfdxg1Ig_iZH3MPJOdL64qvctItpOxkCcA.png)

## <b><font color='darkblue'>Code Example of "MCP Python SDK"</font></b>
<font size='3ptx'><b>The [Model Context Protocol](https://modelcontextprotocol.io/introduction) allows applications to provide context for LLMs in a standardized way, separating the concerns of providing context from the actual LLM interaction</b></font>. [**This Python SDK**](https://github.com/modelcontextprotocol/python-sdk) implements the full MCP specification, making it easy to:
* Build MCP clients that can connect to any MCP server
* Create MCP servers that expose resources, prompts and tools
* Use standard transports like stdio and SSE
* Handle all MCP protocol messages and lifecycle events

### <b><font color='darkgreen'>Core Concepts</font></b>

#### <b><font size='3ptx'>Server</font></b>
The FastMCP server is your core interface to the MCP protocol. It handles connection management, protocol compliance, and message routing:

```python
from dataclasses import dataclass

from fake_database import Database  # Replace with your actual DB type

from mcp.server.fastmcp import Context, FastMCP

# Create a named server
mcp = FastMCP("My App")

# Specify dependencies for deployment and development
mcp = FastMCP("My App", dependencies=["pandas", "numpy"])


@dataclass
class AppContext:
    db: Database


@asynccontextmanager
async def app_lifespan(server: FastMCP) -> AsyncIterator[AppContext]:
    """Manage application lifecycle with type-safe context"""
    # Initialize on startup
    db = await Database.connect()
    try:
        yield AppContext(db=db)
    finally:
        # Cleanup on shutdown
        await db.disconnect()


# Pass lifespan to server
mcp = FastMCP("My App", lifespan=app_lifespan)
```

#### <b><font size='3ptx'>Resources</font></b>
Resources are how you expose data to LLMs. They're similar to GET endpoints in a REST API - they provide data <b>but shouldn't perform significant computation or have side effects</b>:

```python
@mcp.resource("config://app")
def get_config() -> str:
    """Static configuration data"""
    return "App configuration here"


@mcp.resource("users://{user_id}/profile")
def get_user_profile(user_id: str) -> str:
    """Dynamic user data"""
    return f"Profile data for user {user_id}"
```

#### <b><font size='3ptx'>Tools</font></b>
Tools let LLMs take actions through your server. Unlike resources, tools are expected to perform computation and have side effects:

```python
@mcp.tool()
def calculate_bmi(weight_kg: float, height_m: float) -> float:
    """Calculate BMI given weight in kg and height in meters"""
    return weight_kg / (height_m**2)


@mcp.tool()
async def fetch_weather(city: str) -> str:
    """Fetch current weather for a city"""
    async with httpx.AsyncClient() as client:
        response = await client.get(f"https://api.weather.com/{city}")
        return response.text
```

#### <b><font size='3ptx'>Prompts</font></b>
Prompts are reusable templates that help LLMs interact with your server effectively:

```python
@mcp.prompt()
def review_code(code: str) -> str:
    return f"Please review this code:\n\n{code}"


@mcp.prompt()
def debug_error(error: str) -> list[base.Message]:
    return [
        base.UserMessage("I'm seeing this error:"),
        base.UserMessage(error),
        base.AssistantMessage("I'll help debug that. What have you tried so far?"),
    ]
```

#### <b><font size='3ptx'>Images</font></b>
FastMCP provides an <b><font color='blue'>Image</font></b> class that automatically handles image data:

```python
@mcp.tool()
def create_thumbnail(image_path: str) -> Image:
    """Create a thumbnail from an image"""
    img = PILImage.open(image_path)
    img.thumbnail((100, 100))
    return Image(data=img.tobytes(), format="png")
```

#### <b><font size='3ptx'>Context</font></b>
The Context object gives your tools and resources access to MCP capabilities:

```python
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("My App")


@mcp.tool()
async def long_task(files: list[str], ctx: Context) -> str:
    """Process multiple files with progress tracking"""
    for i, file in enumerate(files):
        ctx.info(f"Processing {file}")
        await ctx.report_progress(i, len(files))
        data, mime_type = await ctx.read_resource(f"file://{file}")
    return "Processing complete"
```

### <b><font color='darkgreen'>Installation</font></b>
For projects using pip for dependencies:

In [2]:
# pip install mcp mcp[cli]
!pip freeze | grep mcp

mcp==1.6.0


<a id='quick_start_mcp_server_with_client'></a>
### <b><font color='darkgreen'>Quickstart (MCP server & client)</font></b>
Let's create a simple MCP server that exposes a calculator tool and some data:

- `mcp_server.py`
```python
from mcp.server.fastmcp import FastMCP

# Create an MCP server
mcp = FastMCP("Demo")


# Add an addition tool
#### Tools ####
# Add an addition tool
@mcp.tool()
def add(a: int, b: int) -> int:
  """Add two numbers"""
  print(f"Adding {a} and {b}")
  return a + b


# Add a dynamic greeting resource
@mcp.resource("greeting://{name}")
def get_greeting(name: str) -> str:
  """Get a personalized greeting"""
  return f"Hello, {name}!"


if __name__ == "__main__":
    # Initialize and run the server
  mcp.run(transport='sse')
```

For MCP server's transport (`sse` or `stdio`), you could refer to [here](https://modelcontextprotocol.io/docs/concepts/transports#built-in-transport-types) for details:
> Transports in the Model Context Protocol (MCP) provide the foundation for communication between clients and servers. A transport handles the underlying mechanics of how messages are sent and received.

The common two options as are:
* **Standard Input/Output (stdio)**: The stdio transport enables communication through standard input and output streams. This is particularly useful for local integrations and command-line tools.
  - Building command-line tools
  - Implementing local integrations
  - Needing simple process communication
  - Working with shell scripts
* **Server-Sent Events (SSE)**: SSE transport enables server-to-client streaming with HTTP POST requests for client-to-server communication.
  - Only server-to-client streaming is needed
  - Working with restricted networks
  - Implementing simple updates

Then We can run it by below command:
```shell
$ python mcp_server.py
INFO:     Started server process [4100]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

Next, we could prepare another MCP client to interact with our MCP server:

* `mcp_client.py`
```python
from mcp import ClientSession
from mcp.client.sse import sse_client


async def run():
  async with sse_client(url="http://localhost:8000/sse") as streams:
    async with ClientSession(*streams) as session:
      await session.initialize()

      # List available tools
      print('===== List available tools =====')
      tools = await session.list_tools()
      print(str(tools) + '\n')

      # Call a tool
      print('===== Call a tool =====')
      result = await session.call_tool("add", arguments={"a": 4, "b": 5})
      print(result.content[0].text + '\n')

      # List available resources
      print('===== List resources =====')
      resources = await session.list_resources()
      print("resources", resources)
      print("")

      # Read a resource
      print('===== Read a resource =====')
      content = await session.read_resource("greeting://john")
      print("content", content.contents[0].text)
      print("")


if __name__ == "__main__":
  import asyncio

  asyncio.run(run())
```

We could execute it as below:
```shell
$ ./mcp_client.py
===== List available tools =====
meta=None nextCursor=None tools=[Tool(name='add', description='Add two numbers', inputSchema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'})]

===== Call a tool =====
9

===== List resources =====
resources meta=None nextCursor=None resources=[Resource(uri=AnyUrl('resource://some_static_resource'), name='resource://some_static_resource', description=None, mimeType='text/plain', size=None, annotations=None)]

===== Read a resource =====
content Hello, john!
```

## <b><font color='darkblue'>LangChain + MCP</font></b>
([YT:Using MCP with LangGraph agents](https://www.youtube.com/watch?v=OX89LkTvNKQ)) <font size='3ptx'><b>Anthropic's Model Context Protocol (MCP) is an open source protocol to connect LLMs with context, tools, and prompts</b>. It has a growing number of "servers" for connecting to various tools or data sources. Here, we show how to connect any MCP server to LangGraph agents, and use MCP tools. </font> ([Repo: langchain-mcp-adapters](https://github.com/langchain-ai/langchain-mcp-adapters))

![ui](images/1.png)

### <b><font color='darkgreen'>Installation</font></b>
The sample code below is coming from Github repo [**langchain-mcp-adapters**](https://github.com/langchain-ai/langchain-mcp-adapters), we have to install it first:
> This library provides a lightweight wrapper that makes Anthropic [**Model Context Protocol**](https://modelcontextprotocol.io/introduction) (MCP) tools compatible with [**LangChain**](https://github.com/langchain-ai/langchain) and [**LangGraph**](https://github.com/langchain-ai/langgraph).

In [6]:
# pip install langchain-mcp-adapters
!pip freeze | grep -P 'langchain-mcp-adapters|langchain'

langchain-core==0.3.49
langchain-mcp-adapters==0.0.5


### <b><font color='darkgreen'>MCP Server</font></b>
Here we implement an simple the MCP server **`math_mcp_server.py`** to handle math related problem:
- `math_mcp_server.py`:
```python
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")
```

### <b><font color='darkgreen'>MCP Client</font></b>
To use the MCP client below using OpenAI LLM, you have to apply/generate an OpenAPI key first [here](https://platform.openai.com/api-keys) and use environment variable `OPENAI_API_KEY` to store it for reference. Below MCP client will use OpenAI to answer a simple math question `what's (3 + 5) x 12?` by leveraging tools in our MCP server:

- **`mcp_client_with_math_problem.py`**:
```python
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["/path/to/math_mcp_server.py"],
)

async def main():
  async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
      # Initialize the connection
      await session.initialize()

      # Get tools
      tools = await load_mcp_tools(session)

      # Create and run the agent
      agent = create_react_agent(model, tools)
      agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
      print('Agent response:')
      for message in agent_response['messages']:
        print(f'{message.__class__.__name__}: {message}')
        print("")


# Run the async function
asyncio.run(main())
```

From the code, it will leverage function [`create_react_agent`](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) to create an ReAct agent:
> The **LLM ReAct (Reasoning + Acting) framework is an approach that combines Large Language Models (LLMs) with reasoning and action-taking capabilities**. This allows AI agents to think, decide, and execute tasks autonomously rather than just generating static responses.


Then below is the execution result:
```shell
$ ./mcp_client_with_math_problem.py
Agent response:
HumanMessage: content="what's (3 + 5) x 12?" additional_kwargs={} response_metadata={} id='e358bf94-ce8f-4f82-92be-915806b9275d'

AIMessage: content='' additional_kwargs={'tool_calls': [{'id': 'call_AFOWozah4ZwBchA50hypk5ke', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_nqxKzHmYvCZugl46rvfXTWx4', 'function': {'arguments': '{"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 77, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGhpH5pncUiSVfWA73PXyFigjNz2v', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0bf5927c-9005-4e96-abd0-5e191e457111-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_AFOWozah4ZwBchA50hypk5ke', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 8, 'b': 12}, 'id': 'call_nqxKzHmYvCZugl46rvfXTWx4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 77, 'output_tokens': 51, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

ToolMessage: content='8' name='add' id='4bbdcbcc-6ebd-47fb-9407-cca308a9bf1f' tool_call_id='call_AFOWozah4ZwBchA50hypk5ke'

ToolMessage: content='96' name='multiply' id='aca35ceb-717a-48dd-918e-29450ef47308' tool_call_id='call_nqxKzHmYvCZugl46rvfXTWx4'

AIMessage: content='The result of \\((3 + 5) \\times 12\\) is 96.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 143, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGhpJRbfmopZOraLa2wqbf4nAi4AV', 'finish_reason': 'stop', 'logprobs': None} id='run-ae7bed58-8147-49a6-ae06-b180a5cb137f-0' usage_metadata={'input_tokens': 143, 'output_tokens': 22, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
```

### <b><font color='darkgreen'>Multiple MCP Servers</font></b>
The [**library**](https://github.com/langchain-ai/langchain-mcp-adapters?tab=readme-ov-file#multiple-mcp-servers) also allows you to connect to multiple MCP servers and load tools from them. Here we prepared a second MCP server to hanlde weather related questions:
* `weather_mcp_server.py`
```python
from typing import List
from mcp.server.fastmcp import FastMCP


mcp = FastMCP("Weather")


@mcp.tool()
async def get_weather(location: str) -> str:
  """Get weather for location."""
  return "It's always sunny in Taiwan"


if __name__ == "__main__":
  mcp.run(transport="sse")
```

Let's start the Weather MCP server by executing it:
```shell
$ python weather_mcp_server.py
```

Let's create another MCP client to include both Math and Weather MCP servers:
- `mcp_client_with_both_math_and_weather_problem.py`:
```python
# Create server parameters for stdio connection
import asyncio

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")


MATH_MCP_SERVER_MODULE_PATH = (
    "/home/john/Gitrepos/ml_articles/medium/Clear_intro_to_mcp/math_mcp_server.py")

async def main():
  async with MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": [MATH_MCP_SERVER_MODULE_PATH],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
  ) as client:
    agent = create_react_agent(model, client.get_tools())
    for question in [
          "what's (3 + 5) x 12?",
          "what is the weather in Taiwan?",
      ]:
      response = await agent.ainvoke({"messages": question})
      print(f'===== Q: {question} =====')
      for message in response['messages']:
        print(f'{message.__class__.__name__}: {message}')
        print("")


# Run the async function
asyncio.run(main())
```

Finally, let's execute it and see how it goes:
```shell
$ ./mcp_client_with_both_math_and_weather_problem.py
===== Q: what's (3 + 5) x 12? =====
HumanMessage: content="what's (3 + 5) x 12?" additional_kwargs={} response_metadata={} id='d5e18352-36af-435b-8316-f46adbba7f4b'

AIMessage: content='' additional_kwargs={'tool_calls': [{'id': 'call_T3qkWNjmZTknweaeHKjYXfoL', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 96, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGi2MVKfDNX3FSjkor5uGKQHq49eL', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-93181dcd-b5df-4a16-a315-48e2ea9ae304-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_T3qkWNjmZTknweaeHKjYXfoL', 'type': 'tool_call'}] usage_metadata={'input_tokens': 96, 'output_tokens': 18, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

ToolMessage: content='8' name='add' id='64385ecf-b886-4e80-8052-4fbcfc6a27aa' tool_call_id='call_T3qkWNjmZTknweaeHKjYXfoL'

AIMessage: content='' additional_kwargs={'tool_calls': [{'id': 'call_kb2gEqwQ0ZFGj2fV10GZcSVs', 'function': {'arguments': '{"a":8,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 121, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGi2NZETDbejIQ2YllWwIGUnIKFbW', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-79caf813-7905-42f3-85d7-9aefddd21be9-0' tool_calls=[{'name': 'multiply', 'args': {'a': 8, 'b': 12}, 'id': 'call_kb2gEqwQ0ZFGj2fV10GZcSVs', 'type': 'tool_call'}] usage_metadata={'input_tokens': 121, 'output_tokens': 18, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

ToolMessage: content='96' name='multiply' id='7f189bef-879b-4d8e-ba3c-8944cac24e3c' tool_call_id='call_kb2gEqwQ0ZFGj2fV10GZcSVs'

AIMessage: content='The result of \\((3 + 5) \\times 12\\) is 96.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 146, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGi2OSnR9WPntdosKaSwbASZiyMt6', 'finish_reason': 'stop', 'logprobs': None} id='run-2e0b564d-7f8f-4502-9d35-8e31fcdff1fc-0' usage_metadata={'input_tokens': 146, 'output_tokens': 22, 'total_tokens': 168, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

===== Q: what is the weather in Taiwan? =====
HumanMessage: content='what is the weather in Taiwan?' additional_kwargs={} response_metadata={} id='5d8b5330-7fae-414b-bb4b-80c9ce090ec2'

AIMessage: content='' additional_kwargs={'tool_calls': [{'id': 'call_GW8L6DqAbe5PcmxTFYbIzpMZ', 'function': {'arguments': '{"location":"Taiwan"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 91, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGi2PwLjt9flSEsoW4aAt4HyGcEE0', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f4016f0f-9bdc-47b5-8f84-f53c15a8eb63-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Taiwan'}, 'id': 'call_GW8L6DqAbe5PcmxTFYbIzpMZ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 91, 'output_tokens': 16, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

ToolMessage: content="It's always sunny in Taiwan" name='get_weather' id='f0451b5a-149a-48d8-adb6-1aba43662155' tool_call_id='call_GW8L6DqAbe5PcmxTFYbIzpMZ'

AIMessage: content='The weather in Taiwan is sunny.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 119, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGi2R63RTKtSZYXue7PmZR4U7XKOi', 'finish_reason': 'stop', 'logprobs': None} id='run-4c69bfeb-558b-46f3-8cf9-462fb7133601-0' usage_metadata={'input_tokens': 119, 'output_tokens': 9, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
```

## <b><font color='darkblue'>Google Gemini 2.0 with MCP (Model Context Protocol) Servers</font></b>
([source](https://github.com/philschmid/gemini-samples/blob/main/examples/gemini-mcp-example.ipynb)) <font size='3ptx'><b>Gemini models can be used with MCP server using its native tool calling capabilities. MCP, or Model Context Protocol, is an open standard introduced by Anthropic designed to standardize how AI models like Gemini interact with external tools and data sources.</b> Instead of requiring custom integrations for each tool, MCP provides a structured way for models to access context, such as functions (tools), data sources (resources), or pre-defined prompts. <b>This allows AI agents to securely and efficiently connect with real-world systems and workflows</b>.</font>

<b>MCP server expose their tools via JSON schema definitions, which can be converted to Gemini compatible OpenAPI schema definitions</b>. This allows you to easily use MCP server with Gemini models, below you will example on how to implement this.

You can learn more about Google Search integration with Gemini here:
* [**Gemini API doc - Function Calling with the Gemini API**](https://ai.google.dev/gemini-api/docs/function-calling?lang=python&example=weather)

In [2]:
# install Google GenAI and MCP
#%pip install google-genai mcp 
!pip freeze | grep -P '(google-genai|mcp)'

google-genai==1.2.0
mcp==1.6.0


### <b><font color='darkgreen'>Simple Example on how to use MCP with Gemini's tool calling</font></b>
MCPs can be used with Google DeepMind Gemini by converting the MCP tools into Gemini compatible tools.

In [4]:
from IPython.display import display, Markdown, Latex

from dotenv import load_dotenv, find_dotenv
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os

os.environ['ALLOW_RESET'] = 'TRUE'
_ = load_dotenv(find_dotenv(os.path.expanduser('~/.env')))

client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY")
)  # Replace with your actual API key setup

**<font color='blue'>StdioServerParameters</font>** configures how to start and communicate with the MCP server:
* **`command="npx"`**: Specifies that the server will be run using the npx package runner.
* **`args`**: A list of command-line arguments passed to npx:
  - `"-y"`: Automatically confirms any prompts during the execution of the npx command.
  - `"@openbnb/mcp-server-airbnb"`: This is the npm package name for an Airbnb-specific MCP server. This server likely provides tools related to booking accommodations.
  - `"--ignore-robots-txt"`: An optional argument that might be specific to the Airbnb MCP server.
* **`env=None`**: Specifies that no additional environment variables should be set for the server process.

In [8]:
# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="npx",  # Executable
    args=[
        "-y",
        "@openbnb/mcp-server-airbnb",
        "--ignore-robots-txt",
    ],  # Optional command line arguments
    env=None,  # Optional environment variables
)

* `async with stdio_client(server_params) as (read, write):`: This establishes a connection to the MCP server. stdio_client starts the server process defined by server_params and provides asynchronous read and write streams for communication.
* `async with ClientSession(read, write) as session:`: This creates an MCP client session using the established communication streams. The <b><font color='blue'>ClientSession</font></b> object allows you to interact with the MCP server.
* `prompt = "I want to book an apartment in Paris for 2 nights. 03/28 - 03/30"`: This is the user's query that will be sent to the Gemini model.
* `await session.initialize()`: Initializes the MCP session, allowing the client to understand the server's capabilities.
* `mcp_tools = await session.list_tools()`: This retrieves a list of tools offered by the connected MCP server (<font color='brown'>in this case, the Airbnb server</font>).
* `tools = types.Tool(function_declarations=[...])`: This is the crucial step where MCP tools are converted into Gemini-compatible Tool objects.
  - It iterates through the `mcp_tools.tools` list.
  - For each MCP tool, it creates a function declaration dictionary with the following keys:
      - `"name"`: The name of the MCP tool.
      - `"description"`: The description of the MCP tool.
      - `"parameters"`: The input schema of the MCP tool, defining the expected arguments.
  - These function declaration dictionaries are then used to create a `types.Tool` object, which is the format Gemini expects for function calling.
* `response = client.models.generate_content(...)`: This sends a request to the Gemini model:
  - `model="gemini-2.0-flash"`: Specifies the Gemini model to use (choose one that supports function calling).
  - `contents=prompt`: The user's query.
  - `config=types.GenerateContentConfig(tools=[tools])`: Configures the generation request to include the converted MCP tools in the tools parameter. This tells Gemini about the available functions it can call.
* **Checking for a Function Call:**
  - The code then checks if the Gemini response contains a function call. If it does, it extracts the function name and arguments.
  - `# In a real app, you would call your function here:`: This is a placeholder indicating where you would implement the logic to actually execute the MCP tool based on the function call received from Gemini. This would likely involve using await session.call_tool(function_call.args, arguments=function_call.args).
  - If no function call is found, the code prints the raw text response from Gemini.

In [7]:
async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            prompt = "I want to book an apartment in Paris for 2 nights. 03/28 - 03/30"
            # Initialize the connection
            await session.initialize()
            
            # Get tools from MCP session and convert to Gemini Tool objects
            mcp_tools = await session.list_tools()
            tools = types.Tool(function_declarations=[
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema,
                }
                for tool in mcp_tools.tools
            ])
            
            # Send request with function declarations
            response = client.models.generate_content(
                model="gemini-2.0-flash",  # Or your preferred model supporting function calling
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0.7,
                    tools=[tools],
                ),  # Example other config
            )
        # Check for a function call
        if response.candidates[0].content.parts[0].function_call:
            function_call = response.candidates[0].content.parts[0].function_call
            print(f"Function to call: {function_call.name}")
            print(f"Arguments: {function_call.args}")
            # In a real app, you would call your function here:
            # result = await session.call_tool(function_call.args, arguments=function_call.args)
            # sent new request with function call
        else:
            print("No function call found in the response.")
            print(response.text)
            
await run()

Function to call: airbnb_search
Arguments: {'location': 'Paris', 'checkin': '2024-03-28', 'checkout': '2024-03-30'}


#### <b>How it Works: Bridging the Gap</b>
The core of this process lies in the conversion of MCP tool definitions to Gemini's function declaration format. Here's a breakdown of how this enables the integration:
1. **MCP Tool Discovery**: The mcp-server-airbnb (or any other MCP server) exposes a set of tools with defined names, descriptions, and input schemas (parameters).
2. **Fetching Tool Definitions**: The Python code uses the MCP client (<b><font color='blue'>ClientSession</font></b>) to connect to the MCP server and retrieve the metadata of these available tools using <font color='blue'>session.list_tools()</font>.
3. **Schema Conversion**: The crucial part is the list comprehension that iterates through the MCP tools and extracts their `name`, `description`, and `inputSchema`. These are then structured into a dictionary format that aligns with Gemini's `function_declarations`.
4. **Providing Tools to Gemini**: The <b><font color='blue'>types.Tool</font></b> object, containing the converted function declarations, is included in the tools parameter of the generate_content request to Gemini.
5. **Gemini's Function Calling**: When Gemini processes the user's prompt along with the provided tools, it can recognize if any of the tools are relevant to fulfill the user's request. If so, it will generate a function call in its response, specifying the name of the function to call and the arguments it believes are necessary based on the tool's input schema.
6. **Executing the MCP Tool** (Implementation Required): The commented-out section await session.call_tool(...) highlights the next step in a real application. After receiving a function call from Gemini, your application would use the MCP client to actually execute the corresponding MCP tool with the provided arguments. The result of this execution could then be sent back to Gemini to generate a final response to the user.

### <b><font color='darkgreen'>Full Agentic example with Gemini and Airbnb MCP</font></b>

In [9]:
from dotenv import load_dotenv, find_dotenv
from typing import List
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os


_ = load_dotenv(find_dotenv(os.path.expanduser('~/.env')))

In [17]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
model = "gemini-2.0-flash"

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="npx",  # Executable
    args=[
        "-y",
        "@openbnb/mcp-server-airbnb",
        "--ignore-robots-txt",
    ],  # Optional command line arguments
    env=None,  # Optional environment variables
)

async def agent_loop(prompt: str, client: genai.Client, session: ClientSession):
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Initialize the connection
    await session.initialize()
    
    # --- 1. Get Tools from Session and convert to Gemini Tool objects ---
    mcp_tools = await session.list_tools()
    tools = types.Tool(function_declarations=[
        {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema,
        }
        for tool in mcp_tools.tools
    ])
    
    # --- 2. Initial Request with user prompt and function declarations ---
    response = await client.aio.models.generate_content(
        model=model,  # Or your preferred model supporting function calling
        contents=contents,
        config=types.GenerateContentConfig(
            temperature=0,
            tools=[tools],
        ),  # Example other config
    )
    
    # --- 3. Append initial response to contents ---
    contents.append(response.candidates[0].content)

    # --- 4. Tool Calling Loop ---            
    turn_count = 0
    max_tool_turns = 5
    while response.function_calls and turn_count < max_tool_turns:
        turn_count += 1
        tool_response_parts: List[types.Part] = []

        # --- 4.1 Process all function calls in order and return in this turn ---
        for fc_part in response.function_calls:
            tool_name = fc_part.name
            args = fc_part.args or {}  # Ensure args is a dict
            print(f"Attempting to call MCP tool: '{tool_name}' with args: {args}")

            tool_response: dict
            try:
                # Call the session's tool executor
                tool_result = await session.call_tool(tool_name, args)
                print(f"MCP tool '{tool_name}' executed successfully.")
                if tool_result.isError:
                    tool_response = {"error": tool_result.content[0].text}
                else:
                    tool_response = {"result": tool_result.content[0].text}
            except Exception as e:
                tool_response = {"error":  f"Tool execution failed: {type(e).__name__}: {e}"}
            
            # Prepare FunctionResponse Part
            tool_response_parts.append(
                types.Part.from_function_response(
                    name=tool_name, response=tool_response
                )
            )

        # --- 4.2 Add the tool response(s) to history ---
        contents.append(types.Content(role="user", parts=tool_response_parts))
        print(f"Added {len(tool_response_parts)} tool response parts to history.")

        # --- 4.3 Make the next call to the model with updated history ---
        print("Making subsequent API call with tool responses...")
        response = await client.aio.models.generate_content(
            model=model,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[tools],
            ),  # Keep sending same config
        )
        contents.append(response.candidates[0].content)

    if turn_count >= max_tool_turns and response.function_calls:
        print(f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop.")

    print("MCP tool calling loop finished. Returning final response.")
    # --- 5. Return Final Response ---
    return response

async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "I want to book an apartment in Paris for 2 nights. 03/28 - 03/30"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await agent_loop(prompt, client, session)
            return res

res = await run()

Running agent loop with prompt: I want to book an apartment in Paris for 2 nights. 03/28 - 03/30
Attempting to call MCP tool: 'airbnb_search' with args: {'checkin': '2024-03-28', 'location': 'Paris', 'checkout': '2024-03-30'}
MCP tool 'airbnb_search' executed successfully.
Added 1 tool response parts to history.
Making subsequent API call with tool responses...
MCP tool calling loop finished. Returning final response.


In [18]:
print(res.text)

OK. I found some apartments in Paris for the dates you provided. Here are some of the top listings:

* Guest house in Champs-Élysées: https://www.airbnb.com/rooms/588302148367299476
* Flat in Opéra: https://www.airbnb.com/rooms/827523433116382506
* Flat in Palais-Royal: https://www.airbnb.com/rooms/1178861381392100805
* Flat in Les Halles: https://www.airbnb.com/rooms/17186194
* Room in Grenelle: https://www.airbnb.com/rooms/32705269

I have a lot more options for you. Do any of these look interesting, or would you like me to continue searching?



## <b><font color='darkblue'>Supplement</font></b>
* [MCP Introduction: Get started with the Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)
* [Dev - How to build MCP Servers and Clients from Scratch](https://dev.to/composiodev/how-to-build-mcp-servers-and-clients-from-scratch-4o2f)
* [Gitrepo - Learn-With-Yash-Agrawal/Projects
/MCP_Server_and_Client](https://github.com/oppasource/Learn-With-Yash-Agrawal/tree/main/Projects/MCP_Server_and_Client) ([YT](https://www.youtube.com/watch?v=-WogqfxWBbM))